In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, SimpleRNN, Bidirectional

# Load the datasets
dataset_1 = pd.read_csv('wine_red_lcm_sorted.csv')
dataset_2 = pd.read_csv('winequality-red-normalized.csv')
quality_dataset = pd.read_csv('quality.csv')

# Prepare the input combinations
inputs = [('H11', 'x3', 'x4', 'x5', 'x6', 'x7','x8', 'x10', 'x11'), 
          ('H12', 'x3', 'x4', 'x5', 'x6', 'x7','x8', 'x10', 'x11'), 
          ('H13', 'x3', 'x4', 'x5', 'x6', 'x7','x8', 'x10', 'x11'),
          ('H14', 'x3', 'x4', 'x5', 'x6', 'x7','x8', 'x10', 'x11'), 
          ('H15', 'x3', 'x4', 'x5', 'x6', 'x7','x8', 'x10', 'x11'), 
          ('H16', 'x3', 'x4', 'x5', 'x6', 'x7','x8', 'x10', 'x11')]
# Prepare the output and convert to one-hot encoding
output = to_categorical(quality_dataset['quality2'])

# Define the number of classes
n_classes = 2

def create_model():
    model = Sequential([
        Bidirectional(SimpleRNN(units=16, activation='relu',return_sequences=False),input_shape=(9,1)),
        Dropout(0.1),
        Dense(n_classes, activation='softmax')
    ])

    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  metrics=['accuracy'])
    return model

models = {}  # To store all trained models
all_predictions = {}  # To store all predictions

# Loop over the input combinations, create and train a model for each
for input_tuple in inputs:
    print(f"Training model with inputs {input_tuple}...")

    # Prepare the inputs
    input_data = pd.concat([dataset_1[[input_tuple[0]]], dataset_2[list(input_tuple[1:])]], axis=1)
    input_data = input_data.values.reshape((-1, 9, 1))  # reshape input
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(input_data, output,
                                                        test_size=0.1, 
                                                        random_state=42)

    # Create and train the model
    model = create_model()
    history = model.fit(X_train, y_train,
                        batch_size=16,
                        epochs=300,
                        validation_split=0.1,
                        verbose=2)

    # Evaluate the model on the test set
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test accuracy: {accuracy}\n")

    # Generate predictions on the test set
    predictions = model.predict(X_test)
    predicted_classes = np.argmax(predictions, axis=1)

    # Store the model and predictions
    models[input_tuple] = model
    all_predictions[input_tuple] = predicted_classes  # Store the class predictions

# Initialize an empty DataFrame to store all predictions
all_predictions_df = pd.DataFrame()

# Loop over the input combinations
for input_tuple in inputs:
    print(f"Processing predictions for inputs {input_tuple}...")

    # Retrieve the predicted classes for the current input combination
    predicted_classes = all_predictions[input_tuple]

    # Convert the predictions into a DataFrame column
    predictions_df = pd.DataFrame(predicted_classes, columns=[f'{input_tuple[0]}_{input_tuple[1]}'])

    # Concatenate the current predictions with the previous predictions DataFrame
    all_predictions_df = pd.concat([all_predictions_df, predictions_df], axis=1)

# Save the all predictions DataFrame to a CSV file
all_predictions_df.to_csv('lcm_all_predictions_birnn.csv', index=False)


In [ ]:
all_predictions_df.to_csv('ave_all_predictions_birnn.csv', index=False)

In [ ]:
import pandas as pd

# 转化y_test为标签编码并导出
y_test_labels = np.argmax(y_test, axis=1)

# 转换为 pandas DataFrame
y_test_df = pd.DataFrame(y_test_labels, columns=['Label'])

# 保存为 CSV 文件
y_test_df.to_csv('y_test_labels.csv', index=False)


In [ ]:
inputs = [('H11', 'x3', 'x4', 'x5', 'x6', 'x7','x8', 'x10', 'x11'), 
          ('H12', 'x3', 'x4', 'x5', 'x6', 'x7','x8', 'x10', 'x11'), 
          ('H13', 'x3', 'x4', 'x5', 'x6', 'x7','x8', 'x10', 'x11'),
          ('H14', 'x3', 'x4', 'x5', 'x6', 'x7','x8', 'x10', 'x11'), 
          ('H15', 'x3', 'x4', 'x5', 'x6', 'x7','x8', 'x10', 'x11'), 
          ('H16', 'x3', 'x4', 'x5', 'x6', 'x7','x8', 'x10', 'x11')]
inputs =[('H1', 'x3', 'x4', 'x5', 'x6', 'x7','x8', 'x10', 'x11')]